In [1]:
import numpy as np
import pandas as pd
import pyfixest as pf
from IPython.display import display, FileLink
import pathlib, requests
from tabout import MTable,  DTable, ETable, BTable, import_dta, get_var_labels, set_var_labels

df = import_dta("https://www.stata-press.com/data/r18/auto.dta")

In [2]:
get_var_labels(df)

{'make': 'Make and model',
 'price': 'Price',
 'mpg': 'Mileage (mpg)',
 'rep78': 'Repair record 1978',
 'headroom': 'Headroom (in.)',
 'trunk': 'Trunk space (cu. ft.)',
 'weight': 'Weight (lbs.)',
 'length': 'Length (in.)',
 'turn': 'Turn circle (ft.)',
 'displacement': 'Displacement (cu. in.)',
 'gear_ratio': 'Gear ratio',
 'foreign': 'Car origin'}

In [3]:
DTable(df, vars=["mpg","weight","length"], bycol=["foreign"], stats=["mean_newline_std"],
       counts_row_below=True, hide_stats=True)

In [4]:
DTable(df, vars=["mpg","weight","length"], bycol=["foreign"])

In [15]:
ETable(pf.feols("mpg ~ csw(weight, length)", data=df))

In [ ]:
est1=pf.feols("mpg ~ weight + length", data=df)
ETable(est1)

In [6]:
import statsmodels.formula.api as smf
est2 = smf.ols("mpg ~ weight + length", data=df).fit()
ETable([est2])

In [7]:
from linearmodels import PanelOLS
df1=df.copy()
df1["year"]=1
dfl=df1.set_index(["make","year"])
est3= PanelOLS.from_formula("mpg ~ 1 + weight + length", data=dfl).fit()
ETable(est3)

In [8]:
ETable([est1, est2, est3], model_heads=["pyfixest", "statsmodels", "linearmodels"])

In [9]:
df["foreign_i"] = (df["foreign"] == "Foreign")*1
set_var_labels(df, {"foreign_i": "Foreign (indicator)"})
est_probit = smf.probit("foreign_i ~ weight + length", data=df).fit(disp=False)
ETable([est_probit], model_stats=["N","pseudo_r2","ll"])

In [10]:
# Poisson FE with a simple FE (foreign)
est_pois = pf.fepois("rep78 ~ weight + length + foreign", data=df)

# Show selected stats suitable for Poisson models
ETable(est_pois, model_stats=["N", "deviance"])


In [11]:
import pandas as pd
from linearmodels.datasets import wage_panel
from linearmodels.panel import PanelOLS

data = wage_panel.load()

est1=pf.feols("lwage ~ expersq + married  + union | nr +year", data=data)

data = data.set_index(["nr", "year"])
est2 = PanelOLS.from_formula(
    "lwage ~  expersq + married + union + EntityEffects + TimeEffects",
    data=data,
    drop_absorbed=True
).fit()

ETable([est1, est2], model_stats=["N","r2","r2_within"])